# Libraries

In [6]:
library(fitzRoy)
library(dplyr)
library(jsonlite)
library(lubridate)

# Functions

In [7]:
preprocess_fixture <- function(fixture) {
    # create new column for each state. 
    fixture_clean <- fixture %>% mutate(region = case_when(
        venue %in% c('Adelaide Hills', 'Adelaide Oval', 'Norwood Oval') ~ "SA",
        venue %in% c('M.C.G.', 'Docklands', 'Eureka Stadium', 'Kardinia Park', 'Marvel Stadium', 
                     'GMHBA Stadium', 'Mars Stadium') ~ "VIC",
        venue %in% c('Carrara', 'Gabba', "Cazaly's Stadium", "Riverway Stadium") ~ "QLD",
        venue %in% c('S.C.G.', 'Sydney Showground', 'Stadium Australia') ~ "NSW",
        venue %in% c('Marrara Oval', 'Traeger Park') ~ 'NT',
        venue %in% c('Bellerive Oval', 'York Park', 'University of Tasmania Stadium') ~ "TAS",
        venue %in% c('Manuka Oval', 'UNSW Canberra Oval') ~ 'ACT',
        venue %in% c('Perth Stadium', 'Optus Stadium', 'Subiaco') ~ 'WA',
        venue %in% c('Jiangwan Stadium', 'Adelaide Arena at Jiangwan Stadium') ~ 'CHN',
        TRUE ~ NA_character_  # set NA for all other observations
    ))
    
    fixture_clean$date <- as.Date(fixture_clean$localtime)
    fixture_clean$time <- format(ymd_hms(fixture_clean$localtime), "%H:%M:%S")
    
    fixture_clean$home_win <- ifelse(fixture_clean$hscore > fixture_clean$ascore, 1, 0) 
    
    # select specific rows
    fixture_clean <- select(fixture_clean, year, round, date, time, region, venue, hteam, ateam, hscore, ascore, home_win)
    
    return(fixture_clean)
}

# function to check if a column contains NA values.
check_na_column <- function(df, col_name) {
    return(sum(is.na(df[[col_name]])))
}

# Data Cleaning

Start with the cruncher data as it provides a good baseline to work with. 

In [4]:
# cruncher gives some good clean baseline data to work with  
cruncher <- fromJSON("https://thecruncherau.vercel.app/afl/data.json")

ERROR: Error in fromJSON("https://thecruncherau.vercel.app/afl/data.json"): could not find function "fromJSON"


In [44]:
cruncher$home_win <- ifelse(cruncher$hscore > cruncher$ascore, 1, 0) 
head(cruncher_clean)

,year,round,date,time,location,region,hteam,ateam,hscore,ascore,home_win
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>
1,2017,1,23/03/2017,19:20:00,Melbourne,VIC,Carlton,Richmond,89,132,0
2,2017,1,24/03/2017,19:50:00,Melbourne,VIC,Collingwood,Western Bulldogs,86,100,0
3,2017,1,25/03/2017,16:35:00,Moore Park,NSW,Sydney,Port Adelaide,82,110,0
4,2017,1,25/03/2017,16:35:00,Docklands,VIC,St Kilda,Melbourne,90,120,0
5,2017,1,25/03/2017,19:25:00,Carrara,QLD,Gold Coast,Brisbane Lions,96,98,0
6,2017,1,25/03/2017,19:25:00,Melbourne,VIC,Essendon,Hawthorn,116,91,1


Create a new column to represent a home win.

In [8]:
fixture_17 <- fetch_fixture_squiggle(2017)
fixture_18 <- fetch_fixture_squiggle(2018)
fixture_19 <- fetch_fixture_squiggle(2019)
fixture_20 <- fetch_fixture_squiggle(2020)
fixture_21 <- fetch_fixture_squiggle(2021)
fixture_22 <- fetch_fixture_squiggle(2022)
fixture_23 <- fetch_fixture_squiggle(2023)

ℹ No round specified - returning results for all rounds in 2017

ℹ Getting data from https://api.squiggle.com.au/?q=games;year=2017

✔ Getting data from https://api.squiggle.com.au/?q=games;year=2017 ... done



ℹ No round specified - returning results for all rounds in 2018

ℹ Getting data from https://api.squiggle.com.au/?q=games;year=2018

✔ Getting data from https://api.squiggle.com.au/?q=games;year=2018 ... done



ℹ No round specified - returning results for all rounds in 2019

ℹ Getting data from https://api.squiggle.com.au/?q=games;year=2019

✔ Getting data from https://api.squiggle.com.au/?q=games;year=2019 ... done



ℹ No round specified - returning results for all rounds in 2020

ℹ Getting data from https://api.squiggle.com.au/?q=games;year=2020

✔ Getting data from https://api.squiggle.com.au/?q=games;year=2020 ... done



ℹ No round specified - returning results for all rounds in 2021

ℹ Getting data from https://api.squiggle.com.au/?q=games;year=2021

✔ Getting data from

Combine all the rows of each of the dataframes to get data for years 2017 to 2023.

In [9]:
fixture = rbind(fixture_17, fixture_18, fixture_19, fixture_20, fixture_21, fixture_22, fixture_23)

Local time used instead of date as it has the time based on the location of the game. Will split local time into two variables, date and time. Will using venue will create a column for State/region. 

In [10]:
fixture = preprocess_fixture(fixture)

In [11]:
fixture$venue <- as.factor(fixture$venue)  # convert to factor datatype
summary(fixture$venue)

# check if region has NA values, to see if new/old stadiums in use
check_na_column(fixture, 'region')

Adelaide Arena at Jiangwan Stadium                     Adelaide Hills 
                                 2                                  1 
                     Adelaide Oval                     Bellerive Oval 
                               165                                 24 
                           Carrara                   Cazaly's Stadium 
                               102                                  8 
                         Docklands                     Eureka Stadium 
                               233                                  9 
                             Gabba                      GMHBA Stadium 
                               107                                 16 
                  Jiangwan Stadium                      Kardinia Park 
                                 1                                 41 
                            M.C.G.                        Manuka Oval 
                               307                                 12 
                      Marrara Oval                       Mars Stadium 
                                 9                                  3 
                    Marvel Stadium                       Norwood Oval 
                                43                                  2 
                     Optus Stadium                      Perth Stadium 
                                23                                112 
                  Riverway Stadium                             S.C.G. 
                                 1                                 74 
                 Stadium Australia                            Subiaco 
                                 1                                 22 
                 Sydney Showground                       Traeger Park 
                                55                                  6 
    University of Tasmania Stadium                 UNSW Canberra Oval 
                                 8                                  6 
                         York Park 
                                20

[1] 0

In [14]:
fixture$home_win <- ifelse(fixture$hscore > fixture$ascore, 1, 0) 

fixture$hdiff <- fixture$hscore - fixture$ascore

In [15]:
head(fixture)

year,round,date,time,region,venue,hteam,ateam,hscore,ascore,home_win,hdiff
<int>,<int>,<date>,<chr>,<chr>,<fct>,<chr>,<chr>,<int>,<int>,<dbl>,<int>
2017,1,2017-03-26,14:50:00,SA,Adelaide Oval,Adelaide,Greater Western Sydney,147,91,1,56
2017,1,2017-03-23,19:20:00,VIC,M.C.G.,Carlton,Richmond,89,132,0,-43
2017,1,2017-03-24,19:50:00,VIC,M.C.G.,Collingwood,Western Bulldogs,86,100,0,-14
2017,1,2017-03-25,19:25:00,VIC,M.C.G.,Essendon,Hawthorn,116,91,1,25
2017,1,2017-03-26,16:40:00,WA,Subiaco,Fremantle,Geelong,73,115,0,-42
2017,1,2017-03-25,19:05:00,QLD,Carrara,Gold Coast,Brisbane Lions,96,98,0,-2


Write to a csv for further python analysis

In [243]:
write.csv(fixture, file='fixture.csv', row.names=FALSE)